# Homework 2. Content Based News RecSys : CS6365 Fall 2025

Justin Mittereder - G49843234

This approach is using content-based for every recommendation regardless of whether or not it is a cold start user or warm user. 

Going to use SBERT to generate embeddings of news article data and User History embeddings

In [1]:
import pandas as pd
import datetime
import numpy as np

col_names  = ['Impression ID', 'User ID', 'Time', 'History', 'Impressions']
train_behaviors_df = pd.read_csv('data/MINDsmall_train/behaviors.tsv', sep='\t', header=None, names=col_names)
train_behaviors_df.sort_values(by=['User ID', 'Time'], inplace=True)
train_behaviors_df.reset_index(drop=True, inplace=True)
val_behaviors_df = pd.read_csv('data/MINDsmall_dev/behaviors.tsv', sep='\t', header=None, names=col_names)
val_behaviors_df.sort_values(by=['User ID', 'Time'], inplace=True)
val_behaviors_df.reset_index(drop=True, inplace=True)

col_names  = ['News ID', 'Category', 'SubCategory', 'Title', 'Abstract', 'URL', 'Title Entities', 'Abstract Entities']
train_news_df = pd.read_csv('data/MINDsmall_train/news.tsv', sep='\t', header=None, names=col_names)
val_news_df = pd.read_csv('data/MINDsmall_dev/news.tsv', sep='\t', header=None, names=col_names)


In [2]:
#print(train_news_df.head())
val_news_df.head()

,News ID,Category,SubCategory,Title,Abstract,URL,Title Entities,Abstract Entities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N18955,health,medical,Dispose of unwanted prescription drugs during ...,NaN,https://assets.msn.com/labs/mind/AAISxPN.html,"[{""Label"": ""Drug Enforcement Administration"", ...",[]
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."


In [3]:
train_behaviors_df["unix_timestamp"] = pd.to_datetime(
    train_behaviors_df["Time"], 
    format="%m/%d/%Y %I:%M:%S %p"
).astype("int64") // 1_000_000_000

val_behaviors_df["unix_timestamp"] = pd.to_datetime(
    val_behaviors_df["Time"], 
    format="%m/%d/%Y %I:%M:%S %p"
).astype("int64") // 1_000_000_000

earliest_train_time = datetime.datetime.fromtimestamp(train_behaviors_df['unix_timestamp'].min())
latest_train_time = datetime.datetime.fromtimestamp(train_behaviors_df['unix_timestamp'].max())
earliest_val_time = datetime.datetime.fromtimestamp(val_behaviors_df['unix_timestamp'].min())
latest_val_time = datetime.datetime.fromtimestamp(val_behaviors_df['unix_timestamp'].max())

print(f"Training impressions are from between {earliest_train_time} and {latest_train_time} . ")
print(f"Validation impressions are from between {earliest_val_time} and {latest_val_time} . ")


Training impressions are from between 2019-11-08 19:00:19 and 2019-11-14 18:59:13 . 
Validation impressions are from between 2019-11-14 19:00:01 and 2019-11-15 18:58:03 . 


In [4]:
train_news_df.drop(columns = ['URL', 'Title Entities', 'Abstract Entities'], inplace=True)
train_news_df.fillna("''")
val_news_df.drop(columns = ['URL', 'Title Entities', 'Abstract Entities'], inplace=True)
val_news_df.fillna("''")
train_news_df.head()

,News ID,Category,SubCategory,Title,Abstract
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the..."
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re..."


In [5]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)
print("Torch version:", torch.__version__)
print("Built with CUDA:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())

Using device: cuda
Torch version: 2.9.1+cu128
Built with CUDA: 12.8
CUDA available: True


In [32]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2", device="cuda")

def encode_article(row):
    text = f"{row['Category']} {row['SubCategory']} {row['Title']} {row['Abstract']}"
    return model.encode(text)

#train_news_df["embedding"] = train_news_df.apply(encode_article, axis=1)
val_news_df["embedding"] = val_news_df.apply(encode_article, axis=1)

In [ ]:
#Normalize the embeddings to same magnitude so comparisons are based on semantic similarity rather than magnitude
val_news_df["embedding"] = val_news_df["embedding"].apply(
    lambda x: x / np.linalg.norm(x)
)

#output to pkl file to load in later so we don't have to get the embeddings again
val_news_df.to_pickle('data/MINDsmall_dev/news_w_embeddings.pkl')

In [6]:
#load generated embeddings
val_news_df = pd.read_pickle("data/MINDsmall_dev/news_w_embeddings.pkl")
type(val_news_df.loc[0, "embedding"])

numpy.ndarray

In [7]:
#for validation users that have no history, set their History to empty list
val_behaviors_df["History"] = val_behaviors_df["History"].fillna("")
val_behaviors_df['History'] = val_behaviors_df['History'].apply(lambda x: x.split())

In [8]:
val_behaviors_df.head()

,Impression ID,User ID,Time,History,Impressions,unix_timestamp
0,8158,U1,11/15/2019 7:28:25 AM,"[N23571, N58267, N25682, N10646, N32607, N5773...",N14637-0 N20036-1,1573802905
1,71327,U10,11/15/2019 6:08:15 AM,"[N27612, N36699, N64777, N9120, N9803, N57967,...",N33397-0 N46917-0 N11930-0 N58612-0 N47612-0 N...,1573798095
2,5144,U10000,11/15/2019 1:16:49 PM,"[N10059, N46978, N53234, N3345, N3345, N9155, ...",N48740-0 N51470-0 N1952-0 N23675-0 N56969-0 N6...,1573823809
3,41696,U10000,11/15/2019 8:39:38 AM,"[N10059, N46978, N53234, N3345, N3345, N9155, ...",N29393-0 N20036-0 N30290-0 N31958-1 N23513-0 N...,1573807178
4,1307,U10000,11/15/2019 9:54:10 AM,"[N10059, N46978, N53234, N3345, N3345, N9155, ...",N50775-1 N31958-0 N53572-0 N5472-0 N58251-0 N3...,1573811650


Get embeddings for each user that are the average of the embeddings for each article in the user's history

In [9]:
val_article_emb_dict = dict(zip(val_news_df["News ID"], val_news_df["embedding"]))
val_user_embeddings = {}

for user, rows in val_behaviors_df.groupby("User ID"):
    clicked = rows["History"].sum()  # concatenates all lists of clicked items
    clicked_embs = []
    for article_id in clicked: 
        if article_id in val_article_emb_dict: # if we have embedding for this article
            clicked_embs.append(val_article_emb_dict[article_id])

    if len(clicked_embs) == 0: #no clicked articles in user history
        continue
    #print(clicked_embs)
    val_user_embeddings[user] = np.mean(clicked_embs, axis=0) #gets average of the embeddings of all articles in validation user's history

In [10]:
print(val_user_embeddings['U80234']) #user embedding for first user in val set

[-5.51458262e-03  4.63245139e-02 -7.96381210e-04  2.77028768e-03
  1.37047730e-02  7.66954385e-03  6.67026732e-03  1.97074618e-02
  6.11189869e-04  2.05093506e-03  6.48367545e-03 -4.25464846e-03
  1.97407207e-03  1.08957314e-03 -1.51671711e-02 -4.13908251e-03
  1.59170921e-03  1.44603308e-02  1.42396864e-04 -9.45746899e-03
 -2.04007025e-03  1.47522539e-02  1.01323389e-02  1.79862906e-03
 -1.81267802e-02 -1.42668840e-02  1.34705966e-02  1.37066087e-02
  2.32296884e-02 -2.42018662e-02 -1.11863958e-02  1.09727476e-02
 -2.02262192e-03 -1.33708771e-02  2.02393630e-06  6.70100469e-03
  3.84260365e-03  5.09854825e-03 -4.47891019e-02 -8.97620153e-03
  6.66282419e-03 -2.68292893e-02 -1.56661496e-02  1.05211856e-02
 -3.23455944e-03 -4.68572602e-02 -2.54858052e-03 -3.00462125e-03
 -8.65201000e-03 -1.12079773e-02  3.44298268e-03  1.87317934e-02
 -1.49982125e-02 -4.11962112e-03  2.37245746e-02  6.85227709e-03
 -6.00880384e-03 -4.40474879e-03 -1.38596287e-02 -3.81818786e-02
 -5.37634036e-03  2.62713

Get User and article similarity by getting the dot product of both embeddings

In [11]:
#get all item ids
item_ids = list(val_article_emb_dict.keys())
#dictionary that maps item ids as keys and index as values
item_id_to_index = {item_id: idx for idx, item_id in enumerate(item_ids)}
#get all embeddings for each article into a matrix
item_matrix = np.vstack([val_article_emb_dict[i] for i in item_ids])  # shape (N_items, embedding_dim)
# transpose to (embedding_dim, N_items)
item_matrix = item_matrix.T  

In [12]:
user_item_scores = {}

for user, user_emb in val_user_embeddings.items():
    scores = np.dot(user_emb, item_matrix)    # shape (N_items,)
    user_item_scores[user] = scores

In [13]:
val_clicked = {}

for _, row in val_behaviors_df.iterrows():
    user = row["User ID"]
    impressions = row["Impressions"].split()
    clicked_item = [item.split('-')[0] for item in impressions if item.endswith('-1')][0] #get all articles clicked on in impression
    val_clicked[user] = clicked_item

In [14]:
def auc_score(scores, clicked_idx):
    pos_score = scores[clicked_idx]
    neg_scores = np.delete(scores, clicked_idx)
    return np.mean(pos_score > neg_scores)

def mrr_score(scores, clicked_idx):
    # rank is 1 + number of items with strictly higher score
    rank = np.sum(scores > scores[clicked_idx]) + 1
    return 1.0 / rank

def get_topk_indices(scores, k):
    idx = np.argpartition(scores, -k)[-k:]
    idx = idx[np.argsort(scores[idx])[::-1]]
    return idx

def ndcg_at_k(ranked_items, clicked_item, k):
    """Compute nDCG@k for a single user (binary relevance)"""
    try:
        rank_position = ranked_items[:k].index(clicked_item) + 1
        return 1.0 / np.log2(rank_position + 1)
    except ValueError:
        return 0.0



In [15]:
# -------- Evaluation Loop ---------
auc_list = []
mrr_list = []
hit5_list = []
hit10_list = []
ndcg5_list = []
ndcg10_list = []

for user, scores in user_item_scores.items():
    clicked = val_clicked[user]
    clicked_idx = item_id_to_index[clicked]

    # ----- AUC, MRR -----
    auc_list.append(auc_score(scores, clicked_idx))
    mrr_list.append(mrr_score(scores, clicked_idx))

    # ----- Top-k -----
    top10 = get_topk_indices(scores, 10)
    top10_ids = [item_ids[i] for i in top10]

    hit10_list.append(int(clicked in top10_ids))
    ndcg10_list.append(ndcg_at_k(top10_ids, clicked, 10))

    # for @5
    top5_ids = top10_ids[:5]
    hit5_list.append(int(clicked in top5_ids))
    ndcg5_list.append(ndcg_at_k(top5_ids, clicked, 5))
    
# -------- Aggregate results ---------
metrics = {
    "AUC": np.mean(auc_list),
    "MRR": np.mean(mrr_list),
    "HitRate@5": np.mean(hit5_list),
    "HitRate@10": np.mean(hit10_list),
    "nDCG@5": np.mean(ndcg5_list),
    "nDCG@10": np.mean(ndcg10_list)
}

print("Evaluation Metrics:")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

Evaluation Metrics:
AUC: 0.6839
MRR: 0.0020
HitRate@5: 0.0016
HitRate@10: 0.0027
nDCG@5: 0.0011
nDCG@10: 0.0014
